In [2]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

import matplotlib.pyplot as plt
import numpy as np

import copy
from collections import namedtuple
import os,glob
import random
import time

import cv2 
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from natsort import natsorted

from ResNet import Bottleneck, ResNet, ResNet50

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

True

In [2]:
class ImageTransform() :
    def __init__(self) :
        self.data_transform = {
            'train' : transforms.Compose([
                # transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize(mean, std)
            ]),
            'val' : transforms.Compose([
                # transforms.Resize(256),
                # transforms.CenterCrop(resize),
                transforms.ToTensor(),
                # transforms.Normalize(mean, std)
            ])
        }
        
    def __call__(self, img, phase) :
        return self.data_transform[phase](img)

In [3]:
batch_size = 16
red_dir = './new_data_175/red'
yellow_dir = './new_data_175/yellow'
green_dir = './new_data_175/green'

red_images_filepaths = sorted([os.path.join(red_dir, f) for f in os.listdir(red_dir)])
yellow_images_filepaths = sorted([os.path.join(yellow_dir, f) for f in os.listdir(yellow_dir)])
green_images_filepaths = sorted([os.path.join(green_dir, f) for f in os.listdir(green_dir)])

image_filepaths = [*red_images_filepaths, *yellow_images_filepaths, *green_images_filepaths]
correct_images_filepaths = [i for i in image_filepaths if cv2.imread(i) is not None]

random.seed(50)
random.shuffle(correct_images_filepaths)

# train_images_filepaths = correct_images_filepaths[:420] # 140 * 3
# val_images_filepaths = correct_images_filepaths[420:] # 35 * 3
# test_images_filepaths = correct_images_filepaths[420:]
train_images_filepaths = correct_images_filepaths[:420] # 140 * 3
val_images_filepaths = correct_images_filepaths[420:473] # 35 * 3
test_images_filepaths = correct_images_filepaths[473:]

print(len(train_images_filepaths), len(val_images_filepaths), len(test_images_filepaths))

# label = train_images_filepaths[0].split('/')[-2]
# print(label)



420 53 52


In [4]:
class UrinalyDataset(Dataset) :
    def __init__(self, file_list, transform=None, phase='train') :
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self) :
        return len(self.file_list)
    
    
    def __getitem__(self, idx) :
        img_path = self.file_list[idx]
        img = Image.open(img_path)
        img_transformed = self.transform(img, self.phase)
        
        # label = img_path.split('/')[-1].split('.')[0]
        label = img_path.split('/')[-2]
        if label == 'red' :
            label = 2
            
        elif label == 'yellow' :
            label = 1

        elif label == 'green' :
            label = 0
            
        return img_transformed, label

In [5]:
train_dataset = UrinalyDataset(train_images_filepaths, transform=ImageTransform(),
                                 phase='train')
val_dataset = UrinalyDataset(val_images_filepaths, transform=ImageTransform(),
                               phase='val')
index = 0 

train_iterator = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_iterator = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloader_dict = {'train' : train_iterator, 'val' : valid_iterator}

batch_iterator = iter(train_iterator)
inputs, label = next(batch_iterator)

# print(inputs.size())
# print(label)


In [6]:
Model = ResNet50(num_classes=3, channels=1).to(device)
optimizer = optim.Adam(Model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)

In [7]:
def calculate_topk_accuracy(y_pred, y, k=2) :
    with torch.no_grad() :
        batch_size = y.shape[0]
        _, top_pred = y_pred.topk(k, 1)
        top_pred = top_pred.t()
        correct = top_pred.eq(y.view(1, -1).expand_as(top_pred))
        correct_1 = correct[:1].reshape(-1).float().sum(0, keepdim=True)
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)        
        acc_1 = correct_1 / batch_size
        acc_k = correct_k / batch_size
    
    return acc_1, acc_k

In [8]:
def train(model, iterator, optimizer, criterion, device) :
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0 
    model.train()
    for (x, y) in iterator :
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc_1 += acc_1.item()
        epoch_acc_5 += acc_5.item()
        
    epoch_loss /= len(iterator)
    epoch_acc_1 /= len(iterator)    
    epoch_acc_5 /= len(iterator)  
    
    return epoch_loss, epoch_acc_1, epoch_acc_5

In [9]:
def evaluate(model, iterator, criterion, device) :
    epoch_loss = 0
    epoch_acc_1 = 0
    epoch_acc_5 = 0
    
    model.eval()
    with torch.no_grad() :
        for (x, y) in iterator :
            x = x.to(device)
            y = y.to(device)
            y_pred = model(x)
            loss = criterion(y_pred, y)
            
            acc_1, acc_5 = calculate_topk_accuracy(y_pred, y)
            epoch_loss += loss.item()
            epoch_acc_1 += acc_1.item()
            epoch_acc_5 += acc_5.item()
            
        epoch_loss /=len(iterator)
        epoch_acc_1 /= len(iterator)
        epoch_acc_5 /= len(iterator)
        
        return epoch_loss, epoch_acc_1, epoch_acc_5

In [10]:
def epoch_time(start_time, end_time) :
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [11]:
## Train Model 

best_valid_acc_1 = 0
epochs = 30

for epoch in range(epochs) :
    start_time = time.monotonic()
    
    train_loss, train_acc_1, train_acc_5 = train(Model, train_iterator, optimizer,
                                                criterion, device)
    valid_loss, valid_acc_1, valid_acc_5 = evaluate(Model, valid_iterator, criterion,
                                                   device)
    
    if valid_acc_1 > best_valid_acc_1 :
        best_valid_acc_1 = valid_acc_1
        torch.save(Model.state_dict(), './experiment/'+str(round(best_valid_acc_1,3))+'_'+str(epoch)+'ResNet-model.pt')
        model_lists = natsorted(glob.glob('./experiment/' + '*'), reverse=True)
        # while len(model_lists) < 9:
        #     os.remove(model_lists[-1])
        #     model_lists = natsorted(glob.glob('./experiment/' + '*'))

        
    end_time = time.monotonic()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch : {epoch+1:02} | Epoch Time : {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss : {train_loss:.3f} | Train Acc @1 : {train_acc_1*100:6.2f}% | Train Acc @5 : {train_acc_5*100:6.2f}%')    
    print(f'\tValid Loss : {valid_loss:.3f} | Valid Acc @1 : {valid_acc_1*100:6.2f}% | Valid Acc @5 : {valid_acc_5*100:6.2f}%')   

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch : 01 | Epoch Time : 6m 26s
	Train Loss : 0.968 | Train Acc @1 :  52.08% | Train Acc @5 :  83.33%
	Valid Loss : 3.431 | Valid Acc @1 :  41.25% | Valid Acc @5 :  79.38%
Epoch : 02 | Epoch Time : 6m 17s
	Train Loss : 0.678 | Train Acc @1 :  67.36% | Train Acc @5 :  96.99%
	Valid Loss : 0.674 | Valid Acc @1 :  71.56% | Valid Acc @5 :  90.62%
Epoch : 03 | Epoch Time : 6m 15s
	Train Loss : 0.429 | Train Acc @1 :  81.94% | Train Acc @5 :  97.92%
	Valid Loss : 0.304 | Valid Acc @1 :  90.62% | Valid Acc @5 : 100.00%
Epoch : 04 | Epoch Time : 6m 17s
	Train Loss : 0.344 | Train Acc @1 :  87.04% | Train Acc @5 :  99.07%
	Valid Loss : 0.443 | Valid Acc @1 :  85.94% | Valid Acc @5 :  95.31%
Epoch : 05 | Epoch Time : 6m 16s
	Train Loss : 0.243 | Train Acc @1 :  90.74% | Train Acc @5 :  98.84%
	Valid Loss : 6.240 | Valid Acc @1 :  36.25% | Valid Acc @5 :  76.25%
Epoch : 06 | Epoch Time : 6m 17s
	Train Loss : 0.292 | Train Acc @1 :  90.05% | Train Acc @5 :  99.54%
	Valid Loss : 0.382 | Valid Acc 

KeyboardInterrupt: 

In [19]:
## Eval Model

import pandas as pd
id_list = []
pred_list = []
_id = 0

model_dir = './experiment/'
best_model = torch.load(model_dir + natsorted(os.listdir(model_dir))[-1])
print('Model: {} loaded!'.format(natsorted(os.listdir(model_dir))[-1]))
Model.load_state_dict(best_model)

pred_color = 'None'

with torch.no_grad() : 
    for test_path in test_images_filepaths :
        img = Image.open(test_path)
        _id = label = test_path.split('/')[-2]
        transform = ImageTransform()
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)
        
        Model.eval()
        outputs = Model(img)
        # preds = F.softmax(outputs, dim=1)[:, 1].tolist()
        preds = F.softmax(outputs, dim=1)
        argmax_preds = np.argmax(preds).tolist()
        
        if argmax_preds == 2 :
            pred_color = 'red'
        elif argmax_preds == 1 :
            pred_color = 'yellow'
        elif argmax_preds == 0 :
            pred_color = 'green'
            
        id_list.append(_id)
        pred_list.append(pred_color)
        
res = pd.DataFrame({
        'ground_truth' : id_list,
        'pred' : pred_list
})

res.sort_values(by='ground_truth', inplace=True)
res.reset_index(drop=True, inplace=True)

res.to_csv('./ResNet_eval.csv', index=False)
res.head(53)

Model: 0.906_2ResNet-model.pt loaded!


,ground_truth,pred
0,green,green
1,green,green
2,green,green
3,green,green
4,green,green
5,green,red
6,green,green
7,green,green
8,green,green
9,green,green


In [22]:
# Eval Model 2

from preprocessing import crop_resizing_binary

file_name = '01523651_M_70_2021-03-31_40_1(yellow).jpg'

origin_img = cv2.imread(file_name, cv2.IMREAD_GRAYSCALE)
processed_img, _ = crop_resizing_binary(origin_img)

model_dir = './experiment/'
best_model = torch.load(model_dir + natsorted(os.listdir(model_dir))[-1])
print('Model: {} loaded!'.format(natsorted(os.listdir(model_dir))[-1]))
Model.load_state_dict(best_model)

pred_color = 'None'

with torch.no_grad() : 
        img = processed_img
        transform = ImageTransform()
        img = transform(img, phase='val')
        img = img.unsqueeze(0)
        img = img.to(device)
        
        Model.eval()
        outputs = Model(img)
        preds = F.softmax(outputs, dim=1)
        argmax_preds = np.argmax(preds).tolist()
        
        if argmax_preds == 2 :
            pred_color = 'red'
        elif argmax_preds == 1 :
            pred_color = 'yellow'
        elif argmax_preds == 0 :
            pred_color = 'green'

        print("image: ", file_name)
        print("pred: ", pred_color)



Exception : (98, 143)
Model: 0.906_2ResNet-model.pt loaded!
image:  01523651_M_70_2021-03-31_40_1(yellow).jpg
pred:  yellow
